# Phase 1 - Exploration & Cleaning

Initial data audit and preprocessing for the Telco churn dataset.

## Load raw data

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

TRAIN_PATH = RAW_DATA_DIR / "customer_churn_dataset-training-master.csv"
TEST_PATH = RAW_DATA_DIR / "customer_churn_dataset-testing-master.csv"

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
train_df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


## Basic overview

In [2]:
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
train_df.describe(include="all").T

Train shape: (440833, 12)
Test shape: (64374, 12)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CustomerID,440832.0,NaN,NaN,NaN,225398.667955,129531.91855,2.0,113621.75,226125.5,337739.25,449999.0
Age,440832.0,NaN,NaN,NaN,39.373153,12.442369,18.0,29.0,39.0,48.0,65.0
Gender,440832,2,Male,250252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tenure,440832.0,NaN,NaN,NaN,31.256336,17.255727,1.0,16.0,32.0,46.0,60.0
Usage Frequency,440832.0,NaN,NaN,NaN,15.807494,8.586242,1.0,9.0,16.0,23.0,30.0
Support Calls,440832.0,NaN,NaN,NaN,3.604437,3.070218,0.0,1.0,3.0,6.0,10.0
Payment Delay,440832.0,NaN,NaN,NaN,12.965722,8.258063,0.0,6.0,12.0,19.0,30.0
Subscription Type,440832,3,Standard,149128,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Contract Length,440832,3,Annual,177198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Spend,440832.0,NaN,NaN,NaN,631.616223,240.803001,100.0,480.0,661.0,830.0,1000.0


## Schema and missingness

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB


In [4]:
train_nulls = train_df.isna().sum().sort_values(ascending=False)
test_nulls = test_df.isna().sum().sort_values(ascending=False)
pd.DataFrame({"train_nulls": train_nulls, "test_nulls": test_nulls}).head(10)

,train_nulls,test_nulls
CustomerID,1,0
Age,1,0
Gender,1,0
Tenure,1,0
Usage Frequency,1,0
Support Calls,1,0
Payment Delay,1,0
Subscription Type,1,0
Contract Length,1,0
Total Spend,1,0


## Target balance

In [5]:
churn_balance = train_df["Churn"].value_counts(normalize=True).rename("share")
churn_balance

Churn
1.0    0.567107
0.0    0.432893
Name: share, dtype: float64

## Cleaning and feature engineering

In [6]:
# Combine sets so preprocessing is consistent
train_df = train_df.copy()
test_df = test_df.copy()
train_df["dataset"] = "train"
test_df["dataset"] = "test"
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Harmonize column names for easier downstream use
rename_map = {
    "CustomerID": "customerID",
    "Usage Frequency": "UsageFrequency",
    "Support Calls": "SupportCalls",
    "Payment Delay": "PaymentDelay",
    "Subscription Type": "SubscriptionType",
    "Contract Length": "ContractType",
    "Total Spend": "TotalSpend",
    "Last Interaction": "LastInteraction"
}
full_df = full_df.rename(columns=rename_map)

# Enforce numeric types and handle identifiers
numeric_cols = [
    col
    for col in full_df.select_dtypes(include=["number"]).columns.tolist()
    if col not in ["customerID"]
]
for col in numeric_cols:
    full_df[col] = pd.to_numeric(full_df[col], errors="coerce")
    full_df[col] = full_df[col].fillna(full_df[col].median())

if "Churn" in full_df.columns:
    full_df["Churn"] = full_df["Churn"].fillna(0).round().astype(int)
if "customerID" in full_df.columns:
    full_df["customerID"] = full_df["customerID"].round().astype("Int64")

# Map binary categories
if "Gender" in full_df.columns:
    full_df["Gender"] = full_df["Gender"].map({"Male": 1, "Female": 0})
    full_df["Gender"] = full_df["Gender"].fillna(full_df["Gender"].median())

# One-hot encode remaining categoricals (keep identifiers untouched)
categorical_cols = full_df.select_dtypes(include=["object", "category"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ["customerID", "dataset"]]
full_encoded = pd.get_dummies(full_df, columns=categorical_cols, drop_first=True)
full_encoded.head()


,customerID,Age,Gender,Tenure,UsageFrequency,SupportCalls,PaymentDelay,TotalSpend,LastInteraction,Churn,dataset,SubscriptionType_Premium,SubscriptionType_Standard,ContractType_Monthly,ContractType_Quarterly
0,2,30.0,0.0,39.0,14.0,5.0,18.0,932.0,17.0,1,train,False,True,False,False
1,3,65.0,0.0,49.0,1.0,10.0,8.0,557.0,6.0,1,train,False,False,True,False
2,4,55.0,0.0,14.0,4.0,6.0,18.0,185.0,3.0,1,train,False,False,False,True
3,5,58.0,1.0,38.0,21.0,7.0,7.0,396.0,29.0,1,train,False,True,True,False
4,6,23.0,1.0,32.0,20.0,5.0,8.0,617.0,20.0,1,train,False,False,True,False


## Persist cleaned dataset

In [7]:
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = PROCESSED_DIR / "churn_clean.csv"
full_encoded.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH

WindowsPath('D:/Portfolio Projects/Customer Lifetime Value & Retention Intelligence Platform/data/processed/churn_clean.csv')